<a href="https://colab.research.google.com/github/Daniel20912/Daniel20912/blob/main/Chat_Bot_busca_de_documentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U google-generativeai

In [22]:
# Import the Python SDK
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata
api_key = userdata.get("SECRET_KEY")

genai.configure(api_key = api_key)

In [23]:
for i in genai.list_models():
  if "embedContent" in i.supported_generation_methods:
    print(i.name)

models/embedding-001
models/text-embedding-004


In [24]:
# Exemplo de embedding

title = "Esperança na Adversidade"
sample_text = ("Em meio aos desafios da vida, a esperança nos oferece um farol de luz que guia nosso caminho. Como uma vela tremeluzente em meio à escuridão, ela ilumina nosso interior, trazendo coragem e resiliência."
"\n"
"A esperança nos lembra que, mesmo nos momentos mais sombrios, existem sementes de crescimento e renovação. Ela nos impulsiona a seguir em frente, não importa as adversidades que possamos enfrentar. É um bálsamo para nossas almas feridas, um refúgio seguro onde podemos encontrar paz e força."
"\n"
"A esperança não é ingênua ou irrealista. Ela reconhece os desafios que encontramos, mas mantém a crença de que um futuro melhor é possível. É um ato de fé que nos permite abraçar o desconhecido e acreditar que tudo no final dará certo."
"\n"
"Portanto, que a esperança nos sustente em nossas jornadas. Que ela nos inspire a superar obstáculos, alcançando nossos sonhos e aspirações. Pois com esperança, todas as coisas são possíveis.")

embeddings = genai.embed_content(model = "models/embedding-001", content=sample_text, title=title, task_type="RETRIEVAL_DOCUMENT")

print(embeddings)

{'embedding': [-0.022275448, -0.034725077, -0.012351845, -0.010644105, 0.03051068, 0.047007676, -0.04125017, -0.01751664, -0.019049002, 0.090354316, -0.0017423758, 0.004017544, 0.012767283, -0.034708392, -0.050512776, -0.022043347, -0.016221872, 0.018355718, -0.008297651, -0.017036961, -0.025740067, -0.02373822, -0.019166866, -0.009120384, 0.00660438, -0.01875605, 0.03536419, -0.040976416, 0.016518746, 0.049676996, -0.08649663, 0.07087407, -0.048173837, -0.016857162, 0.029088624, -0.089635, 0.01836675, 0.0725769, 0.011615912, -0.024883023, -0.0053103515, -0.014922236, -0.002141517, -0.026902866, -0.0044353474, -0.044021532, 0.023996454, 0.03868611, -0.012130704, -0.08696501, -0.021779468, 0.011008092, 0.042554248, 0.016383594, 0.018748598, -0.037288975, 0.045372296, -0.009975281, 0.017532917, 0.039876975, 0.0018557045, 0.01496754, 0.037289795, 0.025870353, -0.015407008, -0.039453417, -0.022318743, -0.031962253, 0.017943613, 0.041420646, 0.021918258, -0.05574707, 0.0052789436, 0.0255978

In [25]:
#LISTAGEM DOS DOCUMENTOS QUE SERÃO BUSCADOS
DOCUMENT1 = {
    "Titulo": "Esperança Resiliente",
    "Conteúdo": "A esperança é a âncora que nos mantém firmes em meio às tempestades. A resiliência é o escudo que nos protege das adversidades. Juntos, esperança e resiliência criam um caminho para superar desafios."
}

DOCUMENT2 = {
    "Titulo": "Gratidão Gratificante",
    "Conteúdo": "A gratidão transforma momentos comuns em tesouros. Expressar gratidão preenche nossos corações com alegria. A gratidão traz paz e plenitude para nossas vidas."
}

DOCUMENT2 = {
    "Titulo": "Coragem Inspiradora",
    "Conteúdo": "A coragem não é ausência de medo, mas agir apesar dele. A coragem é contagiosa, inspirando outros a superar seus limites. Com coragem, podemos alcançar nossos sonhos e criar um futuro melhor."
}
documents = [DOCUMENT1, DOCUMENT2, DOCUMENT2]

In [26]:
df = pd.DataFrame(documents)
df

,Titulo,Conteúdo
0,Esperança Resiliente,A esperança é a âncora que nos mantém firmes e...
1,Coragem Inspiradora,"A coragem não é ausência de medo, mas agir ape..."
2,Coragem Inspiradora,"A coragem não é ausência de medo, mas agir ape..."


In [27]:
model = "models/embedding-001"

In [28]:
def embed_fn(tittle, text):
  return genai.embed_content(model = model, content=text, title=title, task_type="RETRIEVAL_DOCUMENT") ["embedding"]

In [29]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteúdo"]), axis=1)
df

,Titulo,Conteúdo,Embeddings
0,Esperança Resiliente,A esperança é a âncora que nos mantém firmes e...,"[-0.0022987237, -0.025478583, 0.0077283448, -0..."
1,Coragem Inspiradora,"A coragem não é ausência de medo, mas agir ape...","[-0.040315427, -0.019714238, -0.016760392, -0...."
2,Coragem Inspiradora,"A coragem não é ausência de medo, mas agir ape...","[-0.040315427, -0.019714238, -0.016760392, -0...."


In [30]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta= genai.embed_content(model = model, content=consulta, task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)

  return df.iloc[indice]["Conteúdo"]

In [31]:
consulta = "O que realmente é a coragem?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

A coragem não é ausência de medo, mas agir apesar dele. A coragem é contagiosa, inspirando outros a superar seus limites. Com coragem, podemos alcançar nossos sonhos e criar um futuro melhor.


In [32]:
generation_config = {
    "candidate_count": 1,
    "temperature": 0
}

In [33]:
prompt = f"Reescreva esse texto de uma forma mais descontraída, sem adicionar informações que não façam parte do texto: {trecho}"
model_1_2 = genai.GenerativeModel("gemini-1.0-pro", generation_config=generation_config)
response = model_1_2.generate_content(prompt)
print(response.text)

Coragem não é ser um durão sem medo, é encarar o medo de frente e fazer o que precisa ser feito. É como um vírus legal que se espalha, fazendo com que todo mundo se sinta mais corajoso. Com coragem, podemos dar um gás nos nossos sonhos e construir um futuro da hora!
